In [ ]:
#!pip install pytesseract 
#!pip install opencv 
#!pip install numpy as np
#!pip install pillow
#!pip install postal

In [1]:
import pytesseract 
from PIL import Image, ImageOps
import numpy as np
# from postal.parser import parse_address

# extract image function

In [152]:
# this function extracts text from a reciept image using a url.
def extract_text(url):
    import pytesseract 
    from PIL import Image
    
    """extracts test out of image
    --inputs= image_url """
    #opening the image
    im=Image.open(url)
    grey= im.convert(mode="L")
    #threshold= 160
    #image=grey.point(lambda x: 255 if x > threshold else 0)
    
    #configuration for pytesseract to operate with text extraction
    config = ('-l eng --oem 1 --psm 3')
    text = pytesseract.image_to_string(grey, config=config)
    
    return text

# text extraction Demo 

In [154]:
##
image=(r"images/large-receipt-image-dataset-SRD/1012-receipt.jpg")
text=extract_text(image)
print (text)

#14

HP Pho Ga
8930 Mission Dr. #102
pp ooee 91770
Phone (626)268-9999

 
     

bate: Apr 01, 2019 Time; 05:

Server: Admin

Bill; 59980 Table : 14

1 Pho Ga (Small) 8.00

1 Pho Ga (Large) 9.00

1 Gor Cuon 5.25
“Subtotal 22.25
TAX 211
Total 24.36
8. Service Gharge- 40% 8.90

Total $33 26

Suggested Tip : 15% (3,65)

18% (4.38)

20% (4.87)
Open Time : Apr 01, 2019 04:39PM
Printed By : Admin

Thank you for Coming !



# creating alist of the entire data_set locations and names

In [157]:
import os 

dir=[]
directory=r"images/large-receipt-image-dataset-SRD/"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        dir.append(f)

# extracting the texts from every image to a list

In [159]:
texts=[]
for i in dir :
    text=extract_text(i)
    texts.append(text)

# writing the list to TXT file

In [168]:
with open(r'text.txt', 'w',encoding="utf-8") as fp:
    for item in texts:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


# writing list to CSV file 

In [170]:
import pandas as pd 
df=pd.DataFrame(texts)
df.to_csv('texts.csv')

# boot up camera function (basic)

In [4]:
#open camera 
#add automated conturing to it
#then add automated capture

import cv2

cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    ret, thresh = cv2.threshold(frame, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV) 
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
            rect = cv2.boundingRect(c)
            if rect[2] < 100 or rect[3] < 100: continue
            #print cv2.contourArea(c)
            x,y,w,h = rect
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(frame,'Moth Detected',(x+w+10,y+h),0,0.3,(0,255,0))
            cv2.imshow('Input', frame)
            c = cv2.waitKey(1)
            if c == 27:
                break
    sleep(30)
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

# Camera Function Advanced

In [ ]:
#capture largest contour 
#close 

import cv2
from time import sleep 

cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    #reading camera feed 
    ret, frame = cap.read()
    #reseizing the frame
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    #changing the input to grey
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    blurred = cv2.GaussianBlur(frame, (5, 5), 0)
    #building the rectangle kernel 
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
    dilated = cv2.dilate(blurred, rectKernel)
    #building the a function to detect the edges 
    edged = cv2.Canny(dilated, 100, 200, apertureSize=3)
    #finding the contours 
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #finding the largest contours
    largest_contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
    #selecting the largest contour 
    image_with_largest_contours = cv2.drawContours(edged, largest_contours, -1, (0,255,0), 3)

    largest_contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
    image_with_largest_contours = cv2.drawContours(edged.copy(), largest_contours, -1, (0,255,0), 3)

        
    for c in largest_contours:
        rect = cv2.boundingRect(c)
        if rect[2] < 100 or rect[3] < 100: continue
        #print cv2.contourArea(c)
        x,y,w,h = rect
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(frame,'Moth Detected',(x+w+10,y+h),0,0.3,(0,255,0))
        cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c == 27:
        break
    sleep(35)
    cap.release()
    cv2.destroyAllWindows()
    break 


# need to be done
**add photo snapping function to the camera feed**

